# Chapter 6 Sustainabilty and Engineering: Energy Balance

1. [Introduction: Energy Balance](#1.-Introduction)
2. [Simulation: Energy Balance](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)

## 1. Introduction

### Global Energy Balance Model

### Overview

This model simulates the impact of atmospheric **CO₂ concentration** on **global mean temperature** using a simplified **energy balance approach**. It assumes that the Earth reaches thermal equilibrium when the **incoming solar radiation** equals the **outgoing longwave radiation**, which is affected by greenhouse gases.

The model is interactive, allowing users to explore how changes in:
- **CO₂ concentration**
- **Planetary albedo**
- **Climate sensitivity**

affect the Earth's equilibrium temperature.

---

### Key Equations

### 🔹 Incoming Solar Radiation

$$
Q_{\text{in}} = \frac{S(1 - \alpha)}{4}
$$

Where:
- $( S $) = Solar constant (≈ 1361 W/m²)  
- $( \alpha $) = Planetary albedo (reflectivity)  
- Division by 4 accounts for Earth's spherical geometry

---

### 🔹 Outgoing Longwave Radiation

$$
Q_{\text{out}} = \sigma T^4
$$

Where:
- $( \sigma $) = Stefan-Boltzmann constant (5.67 × 10⁻⁸ W/m²·K⁴)  
- $( T $) = Earth's surface temperature in Kelvin

---

### 🔹 Radiative Forcing from CO₂ (IPCC AR4)

$$
\Delta F = 5.35 \cdot \ln\left(\frac{C}{C_0}\right)
$$

Where:
- $( C $) = Current CO₂ concentration (ppm)  
- $( C_0 $) = Reference CO₂ concentration (typically 280 ppm)  
- $( \Delta F $) = Additional energy trapped due to greenhouse effect

---

### 🔹 Equilibrium Temperature

$$
T = \left( \frac{Q_{\text{in}} - \Delta F}{\sigma} \right)^{1/4}
$$

This equation balances incoming and outgoing energy, adjusted for greenhouse forcing.

---

### Climate Sensitivity

Climate sensitivity represents the **temperature increase** associated with a **doubling of CO₂**. While not explicitly used in the Stefan-Boltzmann calculation, it helps interpret the model’s behavior and calibrate feedbacks.

Typical values range from **1.5°C to 4.5°C** per doubling.

---

### Interactive Parameters

| Parameter         | Description                              |
|------------------|------------------------------------------|
| CO₂ (ppm)         | Atmospheric carbon dioxide concentration |
| Albedo            | Fraction of solar energy reflected       |
| Sensitivity (°C)  | Temperature change per CO₂ doubling      |

---

### Assumptions

- Earth is modeled as a single-layer radiative system  
- Only CO₂ is considered (no CH₄, aerosols, or feedbacks)  
- Climate sensitivity is used as a conceptual guide  
- No temporal dynamics (steady-state only)

---

### References

- IPCC (2007). *Fourth Assessment Report*  
- Kiehl & Trenberth (1997). *Earth’s Annual Global Mean Energy Budget*  
- Hartmann (1994). *Global Physical Climatology*

## 2. Simulation

In [4]:
# 📌 Run this cell first in a Jupyter Notebook
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# 🌞 Constants
S = 1361             # Solar constant (W/m²)
sigma = 5.67e-8      # Stefan-Boltzmann constant (W/m²·K⁴)
CO2_ref = 280        # Reference CO₂ concentration (pre-industrial)

# Sea level rise per °C warming (approximate)
sea_level_per_deg = 0.32  # meters per °C (IPCC estimate)

def equilibrium_temperature(C, alpha=0.3):
    Q_in = S * (1 - alpha) / 4
    delta_F = 5.35 * np.log(C / C0)
    T = ((Q_in + delta_F) / sigma) ** 0.25
    return T

def sea_level_rise(C):
    T0 = equilibrium_temperature(C0)
    T = equilibrium_temperature(C)
    delta_T = T - T0
    rise = delta_T * sea_level_per_deg
    return delta_T, rise


# 🧮 Temperature without greenhouse effect (pure radiative balance)
def temperature_no_greenhouse(albedo):
    incoming = S * (1 - albedo) / 4
    T_kelvin = (incoming / sigma)**0.25
    return T_kelvin - 273.15  # °C

# 🧮 Temperature with greenhouse effect (anchored to observed baseline)
def temperature_with_greenhouse(C, sensitivity):
    T_ref = 15.0  # observed global mean temperature at 280 ppm
    delta_T = sensitivity * np.log(C / CO2_ref) / np.log(2)
    return T_ref + delta_T

# 📊 Interactive function
def update_model(CO2, albedo, sensitivity):
    clear_output(wait=True)

    T_no_ghg = temperature_no_greenhouse(albedo)
    T_with_ghg = temperature_with_greenhouse(CO2, sensitivity)
    delta_T = T_with_ghg - temperature_with_greenhouse(CO2_ref, sensitivity)
    rise = delta_T * sea_level_per_deg

    print(f"🌍 CO₂ Concentration: {CO2:.0f} ppm")
    print(f"🌞 Albedo: {albedo:.2f}")
    print(f"🌡️ Climate Sensitivity: {sensitivity:.2f} °C per doubling")
    print(f"\n🌡️ Temperature WITHOUT Greenhouse Effect: {T_no_ghg:.2f} °C")
    print(f"🌡️ Temperature WITH Greenhouse Effect:    {T_with_ghg:.2f} °C")
    print(f"Estimated warming: {delta_T:.2f} °C")
    print(f"Estimated sea level rise: {rise:.2f} meters")

    # 📈 Plot comparison
    CO2_range = np.linspace(280, 600, 100)
    T_vals = [temperature_with_greenhouse(C, sensitivity) for C in CO2_range]

    plt.figure(figsize=(10, 6))
    plt.plot(CO2_range, T_vals, color='darkred', label='With Greenhouse Effect')
    plt.axhline(T_no_ghg, color='blue', linestyle='--', label='Without Greenhouse Effect')
    plt.axvline(CO2, color='gray', linestyle=':', label=f'Current CO₂ = {CO2:.0f} ppm')
    plt.xlabel('CO₂ Concentration (ppm)')
    plt.ylabel('Global Temperature (°C)')
    plt.title('Global Temperature: With vs Without Greenhouse Effect')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

# 🎛️ Sliders
CO2_slider = widgets.FloatSlider(value=420, min=280, max=600, step=5, description='CO₂ (ppm)')
albedo_slider = widgets.FloatSlider(value=0.3, min=0.2, max=0.4, step=0.01, description='Albedo')
sensitivity_slider = widgets.FloatSlider(value=3.0, min=1.0, max=6.0, step=0.1, description='Sensitivity (°C)')

# 🔄 Display interactive controls
interactive_plot = widgets.interactive(
    update_model,
    CO2=CO2_slider,
    albedo=albedo_slider,
    sensitivity=sensitivity_slider
)

display(interactive_plot)

interactive(children=(FloatSlider(value=420.0, description='CO₂ (ppm)', max=600.0, min=280.0, step=5.0), Float…

## 3. Self-Assessment

### Conceptual Questions

1. **Why does the Earth’s geometry require dividing the solar constant by 4 in the energy balance equation?**  
2. **How does planetary albedo influence the Earth’s equilibrium temperature?**  
3. **What role does the Stefan-Boltzmann law play in modeling outgoing longwave radiation?**  
4. **Why is the logarithmic relationship used in calculating radiative forcing from CO₂?**  
5. **How does climate sensitivity help interpret the impact of CO₂ doubling, even if it’s not directly used in the energy balance equation?**  
6. **What are the limitations of modeling Earth as a single-layer radiative system?**  
7. **How would including feedbacks like water vapor or ice-albedo change the model’s predictions?**

---

### 🌍 Reflective Questions

1. **How did changing CO₂ concentration in the model affect your understanding of climate sensitivity?**  
2. **Which parameter—CO₂, albedo, or sensitivity—had the most dramatic effect on equilibrium temperature in your simulations? Why?**  
3. **Did the model’s simplicity help or hinder your grasp of global energy balance?**  
4. **How might this model be used to communicate climate change concepts to non-scientific audiences?**  
5. **If you were to extend this model, what additional components (e.g., aerosols, methane, ocean heat uptake) would you include and why?**

---

### ❓ Quiz Questions

```{admonition} Quiz 1
**1. What does the term "planetary albedo" refer to?**

- ( ) The amount of CO₂ in the atmosphere  
- (x) The fraction of solar energy reflected by Earth  
- ( ) The temperature of the Earth's surface  
- ( ) The strength of the greenhouse effect  

**Explanation:**  
Albedo measures reflectivity—higher albedo means more sunlight is reflected, cooling the planet.
```

```{admonition} Quiz 2
**2. What is the approximate radiative forcing from doubling CO₂ from 280 ppm to 560 ppm?**

- ( ) 2.5 W/m²  
- (x) 3.7 W/m²  
- ( ) 5.35 W/m²  
- ( ) 7.4 W/m²  

**Explanation:**  
Using the formula